# <span style="color:green"><center>Diplomado en Inteligencia Artificial y Aprendizaje Profundo</center></span>

# <span style="color:red"><center>Introducción a la API funcional de Keras</center></span>

##   <span style="color:blue">Profesores</span>

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 
3. Campo Elías Pardo Turriago, cepardot@unal.edu.co 

##   <span style="color:blue">Asesora Medios y Marketing digital</span>
 

4. Maria del Pilar Montenegro, pmontenegro88@gmail.com 

## <span style="color:blue">Asistentes</span>

5. Oleg Jarma, ojarmam@unal.edu.co 
6. Laura Lizarazo, ljlizarazore@unal.edu.co 

## <span style="color:blue">Contenido</span> 

* [Introducción](#Introducción)
* [Prepara datos de MNIST](#Prepara-datos-de-MNIST)
* [Separa lotes de datos y mezclar el conjunto de datos](#Separa-lotes-de-datos-y-mezcla-el-conjunto-de-datos)
* [API Fucional de Keras. Sub-clases](#API-Funcional-de-Keras.-Sub-clases)
* [Optimizador y función de pérdida](#Optimizador-y-función-de-pérdida)
* [Métricas para medir pérdida y precisión](#Métricas-para-medir-pérdida-y-precisión)
* [Función de entrenamiento: diferenciación automática con tf.GradientTape](#Función-de-entrenamiento:-diferenciación-automática-con-tf.GradientTape)
* [Compila el modelo](#Compila-el-modelo)
* [Summary y Plot Model](#Summary-y-Plot-Model) 
* [Entrenamiento y evaluación del modelo](#Entrenamiento-y-evaluación-del-modelo) 

## <span style="color:blue">Introducción</span>

Este es un notebook de Google Colaboratory. Los programas de Python se executan directamente en tu navegador,  una gran manera de aprender y utilizar TensorFlow. Para poder seguir este tutorial, ejecuta este notebook en Google Colab. Basado en [Tensorflow- quick start expertos](https://www.tensorflow.org/tutorials/quickstart/advanced)


In [48]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
print('Version de Tensorflow = ', tf.__version__)

# Objetos de la API de Keras
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model

# datos mnist
from tensorflow.keras.datasets import mnist

# Manejo de datasets como tensores de tf
dataset = tf.data.Dataset.from_tensor_slices

# Métricas para medir pérdida y precisión
loss_metric     = tf.keras.metrics.Mean
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy

Version de Tensorflow =  2.4.1


## <span style="color:blue">Prepara datos de MNIST</span> 

In [22]:

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.0

# Agrega una dimensión para manejo de canales de imágenes en tensorflow
x_train = x_train[...,tf.newaxis]
x_test = x_test[...,tf.newaxis]

In [23]:
x_test.shape

(10000, 28, 28, 1)

## <span style="color:blue">Separa lotes de datos  y mezclar el conjunto de datos</span>

Usamos *tf.data*.  Revise cuidadosamente el tutorial  [tf.data: compila canalizaciones de entrada de TensorFlow](https://www.tensorflow.org/guide/data)

In [27]:
train_ds = dataset((x_train, y_train)).shuffle(10000).batch(32)

test_ds  = dataset((x_test, y_test)).shuffle(10000).batch(32)

In [28]:
test_ds

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float64, tf.uint8)>

## <span style="color:blue">API Funcional de Keras. Sub-clases</span>

In [ ]:
VAmos a derivar nuestro model de la clase Model de la API funcional de Keras.

In [44]:
class MyModel(Model):
    def __init__(self):
        super(MyModel,self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten= Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10, activation='softmax')
    
    def call(self,x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

# crea una instancia del modelo
model = MyModel()

## <span style="color:blue">Optimizador y función de pérdida</span>

Usaremos la entropía cruzada con varias clases (10 en este caso) y el optimizador Adam. La función *SparseCategoricalCrossentropy* recibe como entrada enteros y construye internamente la [codificación one-hot](https://en.wikipedia.org/wiki/One-hot). Puede usar ls función *CategoricalCrossentropy* si desse ingrear directamente las etiquetas en codificación one-hot.

Asegúrese de entender la *entropía cruzada* y su relación con la *codificación one-hot*. 

En caso de duda, consulte al instructor.

In [36]:
loss_object = SparseCategoricalCrossentropy()

optimizer = Adam()

In [ ]:
Suponga por ejemplo que tiene tres categorías: 1,2,3. Discuta el siguiente fragmento (snippet) de código.
    

In [37]:
y_true = [1, 2]
y_pred = [[0.05, 0.95, 0], [0.1, 0.8, 0.1]]

loss_object(y_true, y_pred).numpy()

1.1769392

Puede verificar que en este caso el resultado se obtien de la siguiente forma. Explique por favor.

In [14]:
import numpy as np

-(np.log(0.95) + np.log(0.1))/2

1.176939193690798

O lo que es lo mismo

In [19]:
-(np.log(y_pred[0][y_true[0]]) + np.log(y_pred[1][y_true[1]]))/len(y_pred)

1.176939193690798

## <span style="color:blue">Métricas para medir pérdida y precisión</span>

Escoge metricas para medir la perdida y exactitud del modelo. Estas metricas acumulan los valores cada epoch y despues imprimen el resultado total.


In [39]:
train_loss = loss_metric(name='train_loss')
train_accuracy = accuracy_metric(name='train_accuracy')

test_loss = loss_metric(name='test_loss')
test_accuracy = accuracy_metric(name='loss_accuracy')


## <span style="color:blue">Función de entrenamiento: diferenciación automática con tf.GradientTape</span>

La función de entrenamiento es decorada con el decorador *@tf.function*  Este decorado compila la función como un grafo de TensofFlow invocable.

In [52]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)
    

## <span style="color:blue">Función de prueba (test)</span>

In [51]:
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

## <span style="color:blue">Entrena el modelo</span>

In [56]:
def fit(train_dataset, test_dataset, epochs):
    for epoch in range(epochs):
        for images, labels in train_dataset:
            train_step(images, labels)
        
        for images, labels in test_dataset:
            test_step(images, labels)
        
        template = 'Epoch {}, Pérdida: {}, Exactitud: {}, Pérdida de prueba: {}, Exactitud de prueba {}'
        print(template.format(epoch+1,
                              train_loss.result(),
                              train_accuracy.result(),
                              test_loss.result(),
                              test_accuracy.result()))
        
        # Reinicia las métricas para el siguiente paso
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()

In [57]:
fit(train_ds, test_ds, epochs = 5)

Epoch 1, Pérdida: 0.13630391657352448, Exactitud: 0.9592499732971191, Pérdida de prueba: 5.388667106628418, Exactitud de prueba 98.25999450683594
Epoch 2, Pérdida: 0.04190303012728691, Exactitud: 0.9872333407402039, Pérdida de prueba: 5.171141624450684, Exactitud de prueba 98.3699951171875
Epoch 3, Pérdida: 0.022309603169560432, Exactitud: 0.9925833344459534, Pérdida de prueba: 5.160597324371338, Exactitud de prueba 98.43999481201172
Epoch 4, Pérdida: 0.013041360303759575, Exactitud: 0.9960500001907349, Pérdida de prueba: 5.817919731140137, Exactitud de prueba 98.31999969482422
Epoch 5, Pérdida: 0.009497422724962234, Exactitud: 0.9969666600227356, Pérdida de prueba: 6.04024600982666, Exactitud de prueba 98.4000015258789
